# Setup and init

In [ ]:
# !pip install -r requirements.txt

In [ ]:
# Add the `src` dir to the search path, so that modules can be imported from there
import sys
from pathlib import Path

if not "src" in sys.path[1][-3:]:
    _src = str(Path(sys.path[0]).joinpath("src"))
    sys.path.insert(1, _src)
# Other third party imports
from icecream import ic

# Select the Area of Interest

Select an area of interest, from the options in the parmeter set, and then generate an AOI GeoDF by selection the relevant wards.
(For testing purposes different city and AOI GeoDFs can be used)

In [ ]:
import set_parameters as params
import ordnancesurvey_data as os_data

# Options as "Gosforth" or "Oxford"
city = params.locality["Gosforth"]

# Download ward boundaries from Ordnance Survey
# Use this to create an Area of Interest GeoDF.
# A smaller AOI can be used for testing purposes
os_data.download_boundaries_file()
# os_data.create_aoi_from_boundaries_file(city)
aoi_gdf = os_data.open_aoi()

# Obtain the data

This section downloads the data required for the analtsis. Where possible it will check for updates and only download if new data is available.
In general the data is organised:
* Raw, unaltered data is saved in the `download_dir` directory.
* Unpacked and suitably transformed data is saved in the `data_dir` directory.

In [ ]:
import openstreetmap_data as osm

# Download OSM data for routing
osm.download_latest_osm_file(city)
# Download residential landuse
osm.download_landuse(aoi_gdf)
# Download the node (as points) of all the streets
osm.download_street_nodes(aoi_gdf)
# Download the address points locations
os_data.download_address_point_file()
os_data.create_address_aoi(aoi_gdf)

# Configure the Routing Engine
This uses the Open Street Routing Machine, to generate routes. This is run using docker, though all of the docker commands are encapsulated in the RoutingEngine class.

In [ ]:
from docker_utils import RoutingEngine

reng = RoutingEngine(params.transport_params, city)
reng.preprocess_routing_data()